# Bab 5: Support Vector Machines (SVM)

### 1. Pendahuluan

Bab ini berfokus pada *Support Vector Machine* (SVM), sebuah model *machine learning* yang sangat kuat dan serbaguna. SVM mampu melakukan klasifikasi linier maupun non-linier, regresi, dan bahkan deteksi *outlier*. Model ini sangat populer dan menjadi salah satu yang wajib dikuasai, terutama untuk masalah klasifikasi yang kompleks dengan dataset berukuran kecil hingga sedang.

Bab ini akan membahas konsep inti SVM, mulai dari:
* Bagaimana SVM bekerja untuk klasifikasi linier (*hard margin* dan *soft margin*).
* Cara menangani data non-linier menggunakan *kernel trick*.
* Penerapan SVM untuk tugas regresi.
* Mekanisme matematika di balik SVM.

---

### 2. Klasifikasi SVM Linier (Linear SVM Classification)

Ide fundamental di balik SVM adalah menemukan "jalan" (*street*) terluas di antara kelas-kelas data yang berbeda. Tujuannya adalah untuk memaksimalkan *margin*, yaitu jarak antara garis pemisah (*decision boundary*) dan *instance* pelatihan terdekat dari setiap kelas. *Instance* yang berada di tepi "jalan" ini disebut **support vectors**.

#### a. Klasifikasi Hard Margin
*Hard margin classification* hanya berfungsi jika data dapat dipisahkan secara linier dengan sempurna dan sangat sensitif terhadap *outlier*.

#### b. Klasifikasi Soft Margin
Untuk mengatasi kelemahan *hard margin*, digunakan model yang lebih fleksibel, yaitu *soft margin classification*. Tujuannya adalah menyeimbangkan antara menjaga "jalan" selebar mungkin dan membatasi pelanggaran margin (*margin violations*), yaitu *instance* yang berada di dalam "jalan" atau bahkan di sisi yang salah.

Di Scikit-Learn, keseimbangan ini dikendalikan oleh *hyperparameter* `C`:
* Nilai `C` yang kecil menghasilkan "jalan" yang lebih lebar tetapi lebih banyak pelanggaran margin.
* Nilai `C` yang besar menghasilkan "jalan" yang lebih sempit tetapi lebih sedikit pelanggaran margin.

Berikut adalah contoh implementasi `LinearSVC` pada dataset Iris:

```python
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = (iris["target"] == 2).astype(np.float64)  # Iris virginica

# Membuat pipeline untuk scaling dan klasifikasi SVM linier
svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge", random_state=42)),
])

svm_clf.fit(X, y)

# Prediksi
print(svm_clf.predict([[5.5, 1.7]]))
# Output: [1.]
```
---

### 3. Klasifikasi SVM Non-Linier (Nonlinear SVM Classification)

Untuk menangani dataset yang tidak dapat dipisahkan secara linier, strategi utamanya adalah dengan menambahkan fitur baru, seperti fitur polinomial. Dengan menambahkan fitur baru, dataset yang tadinya non-linier bisa menjadi dapat dipisahkan secara linier di dimensi yang lebih tinggi.

### a. Fitur Polinomial

Menambahkan fitur polinomial dapat dilakukan dengan PolynomialFeatures dari Scikit-Learn.

```python
from sklearn.datasets import make_moons
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15, random_state=42)

polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C=10, loss="hinge", random_state=42))
])

polynomial_svm_clf.fit(X, y)
```

### b. Kernel Polinomial (Polynomial Kernel)

Menambahkan fitur polinomial bisa sangat lambat jika derajatnya tinggi. Untungnya, SVM menawarkan solusi cerdas yang disebut kernel trick. Kernel trick memungkinkan kita mendapatkan hasil yang sama seolah-olah kita menambahkan banyak fitur polinomial, tanpa benar-benar harus menambahkannya. Ini sangat efisien secara komputasi.

```python
from sklearn.svm import SVC

# Menggunakan kernel polinomial derajat 3
poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
])
poly_kernel_svm_clf.fit(X, y)
```

### c. Kernel Gaussian RBF (Gaussian RBF Kernel)

Metode populer lainnya adalah dengan menambahkan fitur yang dihitung menggunakan fungsi kemiripan (similarity function). Salah satu yang paling umum adalah Gaussian Radial Basis Function (RBF).
$$\phi_{\gamma}(\mathbf{x}, \boldsymbol{\ell}) = \exp(-\gamma ||\mathbf{x} - \boldsymbol{\ell}||^2)$$

Kernel RBF juga dapat digunakan dengan kernel trick di Scikit-Learn. Ini adalah salah satu kernel yang paling kuat dan serbaguna.

* Hyperparameter gamma ($\gamma$): Bertindak seperti parameter regularisasi. Jika model overfitting, coba kurangi nilai gamma. Jika underfitting, coba tingkatkan.
*  Hyperparameter C: Sama seperti pada SVM linier, mengontrol trade-off antara margin dan pelanggaran.

```python
# Menggunakan kernel Gaussian RBF
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
])
rbf_kernel_svm_clf.fit(X, y)
```

**Aturan Praktis Pemilihan Kernel:**

Selalu coba LinearSVC terlebih dahulu. Jika dataset tidak terlalu besar, coba SVC(kernel="rbf"). Jika membutuhkan performa lebih, Anda dapat mencoba kernel lain atau melakukan pencarian hyperparameter.

---

### 4. Regresi SVM (SVM Regression)


SVM juga dapat digunakan untuk tugas regresi. Triknya adalah membalikkan tujuan: alih-alih mencoba menemukan "jalan" terluas di antara dua kelas, regresi SVM mencoba memasukkan sebanyak mungkin instance ke dalam "jalan" (street) sambil membatasi pelanggaran margin. Lebar "jalan" dikendalikan oleh hyperparameter epsilon ($\epsilon$).

### a. Regresi SVM Linier
```python
from sklearn.svm import LinearSVR

# Membuat data acak
np.random.seed(42)
m = 50
X = 2 * np.random.rand(m, 1)
y = (4 + 3 * X + np.random.randn(m, 1)).ravel()

# Melatih model LinearSVR
svm_reg = LinearSVR(epsilon=1.5, random_state=42)
svm_reg.fit(X, y)
```

### b. Regresi SVM Non-Linier

Sama seperti klasifikasi, untuk menangani data regresi non-linier, kita bisa menggunakan model SVM dengan kernel, seperti kernel polinomial.
```python
from sklearn.svm import SVR

# Membuat data kuadratik
m = 100
X = 2 * np.random.rand(m, 1) - 1
y = (0.2 + 0.1 * X + 0.5 * X**2 + np.random.randn(m, 1)/10).ravel()

# Melatih SVR dengan kernel polinomial derajat 2
svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X, y)
```

---

### 5. Di Balik Layar (Under the Hood)

Bagian ini membahas dasar-dasar matematika SVM, termasuk:

* **Fungsi Keputusan dan Prediksi**: Bagaimana SVM membuat prediksi berdasarkan fungsi $w^\intercal \cdot x + b$.
* **Tujuan Pelatihan**: Menjelaskan bagaimana tujuan pelatihan untuk hard margin dan soft margin dirumuskan secara matematis untuk meminimalkan kemiringan (slope) dan pelanggaran margin.
* **Quadratic Programming (QP)**: Masalah optimisasi yang diselesaikan oleh SVM.
* **The Dual Problem**: Sebuah pendekatan alternatif untuk menyelesaikan masalah QP yang memungkinkan penggunaan kernel trick.
* **Kernel**: Penjelasan matematis tentang cara kerja kernel trick.